In [8]:
import numpy as np
import argparse
import time
import cv2 as cv
import mediapipe as mp
import pyautogui as pg
import tensorflow as tf
from tensorflow.keras.models import load_model

pg.FAILSAFE = False

# this is some preliminary work
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

# this is necessary for the "drawing" screen inside the camera angle
margin = 0.2
max_screen_width, max_screen_height = 900, 500# pg.size()

#draw_width_offset = int(0.5 * margin * max_screen_width)
#draw_height_offset = int(0.5 * margin * max_screen_height)

draw_width_offset = 0.5 * margin
draw_height_offset = 0.5 * margin

# The draw window is a subset of the frame such that we can also reach the edges of the drawing board
#draw_window_width, draw_window_height = int((1- margin) * max_screen_width), int((1-margin) * max_screen_height)
draw_window_width, draw_window_height = (1- margin), (1-margin)


# this is necessary for the maximum poisition of the mouse
max_screen_width = max_screen_width - 1
max_screen_height = max_screen_height - 1

# Load the gesture recognizer model
model = load_model('mp_hand_gesture')

# Load class names
f = open('gesture.names', 'r')
classNames = f.read().split('\n')
f.close()
print(classNames)


cap = cv.VideoCapture(0)
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
  while cap.isOpened():
    success, image = cap.read()
    frame_width, frame_height, c = image.shape
    className = ""


    #print(f"x: {frame_width}, y: {frame_height}")
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    results = hands.process(image)

    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
      landmarks = []
      for hand_landmarks in results.multi_hand_landmarks:
        #print(hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x)
        #print(hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y)

        # Firstly, retrieve the coordinates of the fingers on the frame (relative coordinates in range [0,1])
        # the coordinates are calculated with the upper left corner as origin (0,0)

        # in this case it should start at 0
        if hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x > 1:
          finger_coor_x = 0
        elif hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x < 0:
          finger_coor_x = 1
        # get the finger coorindates, adjust x value because pyautogui uses a different method
        #else: finger_coor_x = int((1 - hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x) * max_screen_width)
        else: finger_coor_x = 1 - hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x

        if hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y > 1:
          #finger_coor_y = max_screen_height
          finger_coor_y = 1
        elif hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y < 0:
          finger_coor_y = 0
        #else: finger_coor_y = int(hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y * max_screen_height)
        else: finger_coor_y = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y 


        """

        # finger position is within or outside the window, calculate new coordinates
        if finger_coor_x < draw_width_offset: 
          relative_x = 0
          draw_x = int(relative_x * max_screen_width)
        elif finger_coor_x > max_screen_width - draw_width_offset:
          relative_x = 1
          draw_x = int(relative_x * max_screen_width)
        else:
          relative_x = (finger_coor_x - draw_width_offset) / draw_window_width
          draw_x = int(relative_x * max_screen_width)

        if finger_coor_y < draw_height_offset: 
          relative_y = 0
          draw_y = int(relative_y * max_screen_height)
        elif finger_coor_y > max_screen_height - draw_height_offset:
          relative_y = 1
          draw_y = int(relative_y * max_screen_height)
        else:
          relative_y = (finger_coor_y - draw_height_offset) / draw_window_height
          draw_y = int(relative_y * max_screen_width)
        """

        if finger_coor_x < draw_width_offset: 
          relative_x = 0
          draw_x = relative_x
        elif finger_coor_x > draw_window_width + draw_width_offset:
          relative_x = 1
          draw_x = relative_x 
        else:
          relative_x = (finger_coor_x - draw_width_offset) / draw_window_width
          draw_x = relative_x

        if finger_coor_y < draw_height_offset: 
          relative_y = 0
          draw_y = relative_y
        elif finger_coor_y > draw_window_height + draw_height_offset:
          relative_y = 1
          draw_y = relative_y
        else:
          relative_y = (finger_coor_y - draw_height_offset) / draw_window_height
          draw_y = relative_y

        draw_x, draw_y = int(draw_x * max_screen_width), int(draw_y * max_screen_height)
        
        for lm in hand_landmarks.landmark:
          # print(id, lm)
          lmx = int(lm.x * frame_width)
          lmy = int(lm.y * frame_height)

          landmarks.append([lmx, lmy])
        # Predict gesture
        if len(landmarks) == 21:
          prediction = model.predict([landmarks])
          # print(prediction)
          classID = np.argmax(prediction)
          className = classNames[classID]
    

        print(f"x: {draw_x}, y: {draw_y}")
        #pg.moveTo(draw_x, draw_y)
        if className == "fist":
          pg.click()
        #print(f"finger x: {finger_coor_x}, finger y: {finger_coor_y}")
        #print(f"draw x: {draw_x}, draw y: {draw_y}")

        #print("----------------")
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())

    #cv.putText(image, className, (10, 50), cv.FONT_HERSHEY_SIMPLEX, 
                   #1, (0,0,255), 2, cv.LINE_AA)
    # Flip the image horizontally for a selfie-view display.
    image = cv.flip(image, 1)
    cv.putText(image, f"x: {draw_x}, y: {draw_y}", (10, 50), cv.FONT_HERSHEY_SIMPLEX, 
                   1, (0,0,255), 2, cv.LINE_AA)
    #cv.putText(image, className, (10, 50), cv.FONT_HERSHEY_SIMPLEX, 
                   #1, (0,0,255), 2, cv.LINE_AA)

    cv.imshow('MediaPipe Hands', image)
    if cv.waitKey(1) == ord('q'):
        break
cap.release()

"""
cap = cv.VideoCapture(0)
if not cap.isOpened():
    print("Cannot open camera")
    exit()
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    
    # Display the resulting frame
    cv.imshow('frame', frame)
    key = cv.waitKey(1) & 0xFF
    if cv.waitKey(1) == ord('q'):
        break

# When everything done, release the capture
cap.release()
"""
cv.destroyAllWindows()


['okay', 'peace', 'thumbs up', 'thumbs down', 'call me', 'stop', 'rock', 'live long', 'fist', 'smile']
x: 0, y: 499
x: 0, y: 499
x: 0, y: 499
x: 0, y: 499
x: 0, y: 499
x: 0, y: 499
x: 0, y: 499
x: 0, y: 499
x: 0, y: 499
x: 0, y: 499
x: 0, y: 499
x: 0, y: 499
x: 0, y: 499
x: 0, y: 499
x: 0, y: 499
x: 0, y: 499
x: 0, y: 499
x: 0, y: 499
x: 0, y: 499
x: 449, y: 499
x: 0, y: 499
x: 452, y: 499
x: 0, y: 499
x: 510, y: 472
x: 0, y: 497
x: 515, y: 463
x: 0, y: 499
x: 514, y: 463
x: 0, y: 499
x: 524, y: 463
x: 0, y: 492
x: 521, y: 471
x: 0, y: 490
x: 521, y: 471
x: 0, y: 492
x: 522, y: 499
x: 0, y: 490
x: 437, y: 499
x: 0, y: 495
x: 0, y: 499
x: 0, y: 499
x: 0, y: 499
x: 0, y: 499
x: 0, y: 499
x: 0, y: 499
x: 0, y: 499
x: 0, y: 499
x: 0, y: 499
x: 0, y: 499
x: 0, y: 499
x: 0, y: 499
x: 0, y: 499
x: 0, y: 499
x: 4, y: 499
x: 1, y: 499
x: 3, y: 499
x: 3, y: 499
x: 0, y: 499
x: 0, y: 499
x: 0, y: 499
x: 0, y: 499
x: 0, y: 499
x: 0, y: 499
x: 0, y: 499
x: 1, y: 499
x: 1, y: 499
x: 0, y: 499
x: 2, 